# --- Day 15: Timing is Everything ---

The halls open into an interior plaza containing a large kinetic sculpture. The sculpture is in a sealed enclosure and seems to involve a set of identical spherical capsules that are carried to the top and allowed to bounce through the maze of spinning pieces.

Part of the sculpture is even interactive! When a button is pressed, a capsule is dropped and tries to fall through slots in a set of rotating discs to finally go through a little hole at the bottom and come out of the sculpture. If any of the slots aren't aligned with the capsule as it passes, the capsule bounces off the disc and soars away. You feel compelled to get one of those capsules.

The discs pause their motion each second and come in different sizes; they seem to each have a fixed number of positions at which they stop. You decide to call the position with the slot 0, and count up for each position it reaches next.

Furthermore, the discs are spaced out so that after you push the button, one second elapses before the first disc is reached, and one second elapses as the capsule passes from one disc to the one below it. So, if you push the button at time=100, then the capsule reaches the top disc at time=101, the second disc at time=102, the third disc at time=103, and so on.

The button will only drop a capsule at an integer time - no fractional seconds allowed.

For example, at time=0, suppose you see the following arrangement:

```
Disc #1 has 5 positions; at time=0, it is at position 4.
Disc #2 has 2 positions; at time=0, it is at position 1.
```

If you press the button exactly at time=0, the capsule would start to fall; it would reach the first disc at time=1. Since the first disc was at position 4 at time=0, by time=1 it has ticked one position forward. As a five-position disc, the next position is 0, and the capsule falls through the slot.

Then, at time=2, the capsule reaches the second disc. The second disc has ticked forward two positions at this point: it started at position 1, then continued to position 0, and finally ended up at position 1 again. Because there's only a slot at position 0, the capsule bounces away.

If, however, you wait until time=5 to push the button, then when the capsule reaches each disc, the first disc will have ticked forward 5+1 = 6 times (to position 0), and the second disc will have ticked forward 5+2 = 7 times (also to position 0). In this case, the capsule would fall through the discs and come out of the machine.

However, your situation has more than two discs; you've noted their positions in your puzzle input. **What is the first time you can press the button to get a capsule?**

In [1]:
from tqdm import tqdm
import numpy as np
import utils

utils.get_input(15)

['Disc #1 has 17 positions; at time=0, it is at position 1.',
 'Disc #2 has 7 positions; at time=0, it is at position 0.',
 'Disc #3 has 19 positions; at time=0, it is at position 2.',
 'Disc #4 has 5 positions; at time=0, it is at position 0.',
 'Disc #5 has 3 positions; at time=0, it is at position 0.',
 'Disc #6 has 13 positions; at time=0, it is at position 5.']

In [2]:
disc_lengths = [17, 7, 19, 5, 3, 13]
disc_start_positions = [1, 0, 2, 0, 0, 5]

discs = [["." for _ in range(i)] for i in disc_lengths]
for disc, i in enumerate(disc_start_positions):
    discs[disc][i] = "#"
    
def show_discs(discs=discs):
    for i, disc in enumerate(discs):
        pad = (max(len(d) for d in discs) - len(disc)) // 2
        print(f"d{i} |" + " "*pad + ''.join(disc) + " "*pad + "|")
    print("-"*27)

show_discs()

d0 | .#............... |
d1 |      #......      |
d2 |..#................|
d3 |       #....       |
d4 |        #..        |
d5 |   .....#.......   |
---------------------------


Now a func to roll the discs as per the description:

In [3]:
def rotate(arr, num):
    """shits items in arr by num"""
    return np.roll(arr, num)

def roll_discs(discs=discs, num_rolls=1):
    """moves all the discs forward by num_rolls"""
    return [rotate(d, num_rolls) for d in discs]

show_discs(roll_discs(discs)), show_discs(roll_discs(discs, 2))

d0 | ..#.............. |
d1 |      .#.....      |
d2 |...#...............|
d3 |       .#...       |
d4 |        .#.        |
d5 |   ......#......   |
---------------------------
d0 | ...#............. |
d1 |      ..#....      |
d2 |....#..............|
d3 |       ..#..       |
d4 |        ..#        |
d5 |   .......#.....   |
---------------------------


(None, None)

First up to determine if the capsule will fall through, using lru_cache though not sure if its needed, also this func is pretty computationaly intensive - I can probably hard code the indexes rather than rotating each disc:

In [4]:
from functools import lru_cache

@lru_cache(10000)
def is_solved(discs=discs):
    return all([rotate(d,i+1)[0] == "#" for i,d in enumerate(discs)])

is_solved()

False

Now to put all this together by rotating the discs one timestep at a time and checking if they now solve the puzzle:

In [11]:
def solve(discs=discs):
    t = 0
    max_disc = max(len(d) for d in discs)
    
    print(f"Starting at time {t}:")
    print(f"{show_discs(discs)}")
    
    while True:
        if is_solved((tuple(d) for d in discs)):
            print(f"The capsule should pass through at time {t}")
            show_discs(discs)
            return t
        else:
            discs = roll_discs(discs)
            t += 1 # max_disc
            
            if t % 50000 == 0:
                print(f"passing through time {t}")

%time solve()

Starting at time 0:
d0 | .#............... |
d1 |      #......      |
d2 |..#................|
d3 |       #....       |
d4 |        #..        |
d5 |   .....#.......   |
---------------------------
None
passing through time 50000
passing through time 100000
passing through time 150000
passing through time 200000
passing through time 250000
passing through time 300000
The capsule should pass through at time 317371
d0 | ................# |
d1 |      .....#.      |
d2 |................#..|
d3 |       .#...       |
d4 |        .#.        |
d5 |   .......#.....   |
---------------------------
CPU times: user 1min 50s, sys: 1.11 s, total: 1min 51s
Wall time: 1min 53s


317371

`317371` is the right answer!

# --- Part Two ---

After getting the first capsule (it contained a star! what great fortune!), the machine detects your success and begins to rearrange itself.

When it's done, the discs are back in their original configuration as if it were time=0 again, but a new disc with 11 positions and starting at position 0 has appeared exactly one second below the previously-bottom disc.

**With this new disc, and counting again starting from time=0 with the configuration in your puzzle input, what is the first time you can press the button to get another capsule?**

---

First up, making the new discs:

In [5]:
disc_lengths_2 = [17, 7, 19, 5, 3, 13, 11]
disc_start_positions_2 = [1, 0, 2, 0, 0, 5, 0]

discs2 = [["." for _ in range(i)] for i in disc_lengths_2]
for disc, i in enumerate(disc_start_positions_2):
    discs2[disc][i] = "#"
    
show_discs(discs2)

d0 | .#............... |
d1 |      #......      |
d2 |..#................|
d3 |       #....       |
d4 |        #..        |
d5 |   .....#.......   |
d6 |    #..........    |
---------------------------


My solution above is pretty slow, so by adding a new disc it will be really slow. So I'm writing a faster one:

- **rotating the structure by the length of the longest disc**, which is disc 2 with length 19, taking into account that we first have to move it up to a position where it solves the puzzle, which is 14 rotations for both parts 1 and 2

In [6]:
rotate(discs[2],14)

array(['.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.',
       '.', '.', '.', '#', '.', '.'], dtype='<U1')

In [17]:
disc_lengths = [len(d) for d in discs]
big_disc_index = disc_lengths.index(max(disc_lengths))
biggest_disc = discs[big_disc_index]

biggest_disc.index("#")

2

In [9]:
def solve_faster(discs=discs):
    """solves faster by rotating the discs the length of the longest disc"""
    t = 0
    
    print("The discs initial state is:")
    print(show_discs(discs))
    
    pbar = tqdm()
    
    # bring forward the biggest disc
    discs = roll_discs(discs, 14)
    t += 14
    
    pbar.update(14) # since we have already done 14 rolls
    pbar.write(f"Starting at time {t}:")
    pbar.write(f"{show_discs(discs)}")
    
    while True:
        if is_solved((tuple(d) for d in discs)):
            pbar.close()
            print(f"The capsule will drop through at time {t}, where the discs will look like:")
            show_discs(discs)
            return t
        else:
            discs = roll_discs(discs, 19)
            t += 19
            pbar.update(19)
            
assert solve_faster(discs) == 317371

3833it [00:00, 38249.24it/s]

The discs initial state is:
d0 | .#............... |
d1 |      #......      |
d2 |..#................|
d3 |       #....       |
d4 |        #..        |
d5 |   .....#.......   |
---------------------------
None
Starting at time 14:
d0 | ...............#. |
d1 |      #......      |
d2 |................#..|
d3 |       ....#       |
d4 |        ..#        |
d5 |   ......#......   |
---------------------------
None


317371it [00:07, 44203.00it/s]

The capsule will drop through at time 317371, where the discs will look like:
d0 | ................# |
d1 |      .....#.      |
d2 |................#..|
d3 |       .#...       |
d4 |        .#.        |
d5 |   .......#.....   |
---------------------------


Eureka, thats more than 10x faster than above! the main change here is moving forward 19 rolls at a time instead of 1.

In [10]:
solve_faster(discs2)

2123it [00:00, 21187.99it/s]

The discs initial state is:
d0 | .#............... |
d1 |      #......      |
d2 |..#................|
d3 |       #....       |
d4 |        #..        |
d5 |   .....#.......   |
d6 |    #..........    |
---------------------------
None
Starting at time 14:
d0 | ...............#. |
d1 |      #......      |
d2 |................#..|
d3 |       ....#       |
d4 |        ..#        |
d5 |   ......#......   |
d6 |    ...#.......    |
---------------------------
None


2080951it [00:54, 38495.19it/s]

The capsule will drop through at time 2080951, where the discs will look like:
d0 | ................# |
d1 |      .....#.      |
d2 |................#..|
d3 |       .#...       |
d4 |        .#.        |
d5 |   .......#.....   |
d6 |    ....#......    |
---------------------------


2080951

Part 2 answer: `2080951` for my input

# Notes

- lots of speed up which can be done, like not using arrays
- why am i getting a None output in the solve_faster func?
- rolling arrays isn't very fast, I could be using math instead, like `(if time + disc_position) % length_of_disc == 0` and do a and for all discs